In [ ]:
import numpy as np
import pandas as pd
import glob
import cv2
from ActiveContourDisc import activecontour
from ROI_from_excel import getROI
from ROI_from_excel import ShowResizedIm
from gutDWT import dwtopticdiscfinder
from cupB_eval import find_cup
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [ ]:
def getCDR(img,ROI_size):
    canvas = img.copy()
    #1 img -> preprocess
    #2 proprocess -> coordinate
    xODC,yODC = dwtopticdiscfinder(img)
    xODC = int(xODC)
    yODC = int(yODC)
    #3 coordinate -> ROI
    ROI_image,x_ROI1,y_ROI1,x_ROI2,y_ROI2 = getROI(img,xODC,yODC,ROI_size)
    #4 disc
    canvas,__,top_OD,__,bot_OD,snake = activecontour(canvas,ROI_image,xODC,
                                               yODC,ROI_size,radius=250,B=90,WL=0.123,WE=6.5)
    ROI_imageL,x_ROI1,y_ROI1,x_ROI2,y_ROI2 = getROI(canvas,xODC,yODC,ROI_size)
    
    OD_size = round(bot_OD-top_OD,0)
    print(OD_size)
    #5 cup
    
    center, radious, area, error, image_result = find_cup(ROI_image,ROI_imageL)
    cv2.circle(canvas,(int(center[0]+x_ROI1),int(center[1]+y_ROI1)), radious,(0,255,0),2)
    OC_size = round(radious*2,0)
    #6 CDR
    CDR = round(OC_size/OD_size,2)
    cv2.line(canvas, (xODC+20, yODC), (xODC-20, yODC), (0, 0, 255), 3) 
    cv2.line(canvas, (xODC, yODC+20), (xODC, yODC-20), (0, 0, 255), 3) 
    return canvas,OD_size,OC_size,CDR

In [10]:
#=============USER INPUT=============

crop_size = 500

#=============USER END=============
path = 'RAW_FUNDUS_INPUT/*.jpg'
image_list = []
for filename in glob.glob(path):
    image_list.append(filename)
name_list = []
OD_size_list = []
OC_size_list = []
CDR_list = []
for i in image_list:
    img = cv2.imread(i) #import image
    output,OD_size,OC_size,CDR = getCDR(img,crop_size)
    name_list.append(i)
    OD_size_list.append(OD_size)
    OC_size_list.append(OC_size)
    CDR_list.append(CDR)

    #ShowResizedIm(output,"image_result",2)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
df = pd.DataFrame({'file name':name_list,
                       'OD_size':OD_size_list,
                       'OC_size':OC_size_list,
                       'CDR':CDR_list
                      })
writer = ExcelWriter('CDR.xlsx')
df.to_excel(writer,'Sheet1',index=False)
writer.save()

263.0
(500, 500)
(500, 500)
235.0
(500, 500)
(500, 500)
296.0
(500, 500)
(500, 500)
228.0
(500, 500)
(500, 500)
306.0
(500, 500)
(500, 500)
249.0
(500, 500)
(500, 500)
247.0
(500, 500)
(500, 500)
302.0
(500, 500)
(500, 500)
252.0
(500, 500)
(500, 500)
253.0
(500, 500)
(500, 500)
170.0
(500, 500)
(500, 500)
280.0
(500, 500)
(500, 500)
273.0
(500, 500)
(500, 500)
269.0
(500, 500)
(500, 500)
261.0
(500, 500)
(500, 500)
279.0


IndexError: index 281 is out of bounds for axis 0 with size 281

<Figure size 432x288 with 0 Axes>

In [ ]:
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk,Image
import cv2

In [12]:
#=============USER INPUT=============

crop_size = 500
image_show_size = 4
#=============USER END=============
window = Tk()
window.geometry("1300x700") #Width x Height
window.title('CDR finder')
top_text_frame = Frame(window)
top_text_frame.pack()

image_frame = Frame(window)
image_frame.pack()

bot_result_frame = Frame(window)
bot_result_frame.pack()

label = Label(top_text_frame,text='WELCOME TO CDR FINDER alpha0.01')
label.pack()

label_OD = Label(bot_result_frame,text="OD = ")
label_OD.pack()
label_OC = Label(bot_result_frame,text="OC = ")
label_OC.pack()
label_CDR = Label(bot_result_frame,text="CDR = ")
label_CDR.pack()
label_NP = Label(bot_result_frame,text="")
label_NP.pack()
global image_path 
global imgout
def getPathBtn():
    image_path = ""
    window.filename =  filedialog.askopenfilename(initialdir = "/",
                                                  title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
    image_path = window.filename
    if image_path is not '':
        print(image_path)        
        img = cv2.imread(image_path)
        
        output,OD_size,OC_size,CDR = getCDR(img,crop_size)
        
        img_H,img_W = img.shape[:2]
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        output = cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img, (int(img_W/image_show_size),int(img_H/image_show_size)), interpolation = cv2.INTER_AREA)
        out_resized = cv2.resize(output, (int(img_W/image_show_size),int(img_H/image_show_size)), interpolation = cv2.INTER_AREA)
        img_Tk = ImageTk.PhotoImage(image = Image.fromarray(img_resized))
        out_Tk = ImageTk.PhotoImage(image = Image.fromarray(out_resized))
        label_img = Label(image_frame,image=img_Tk)
        label_out = Label(image_frame,image=out_Tk)
        label_img.image = img_Tk # keep a reference!
        label_out.image = out_Tk # keep a reference!
        label_img.grid(row=0,column=1, sticky="en")
        label_out.grid(row=0,column=2, sticky="en")
        
        label_OD.config(text="OD = "+str(OD_size)+" pixels")
        label_OC.config(text="OC = "+str(OC_size)+"50 pixels")
        label_CDR.config(text="CDR = "+str(CDR))
        if CDR < 0.6:
            label_NP.config(text="Negative(-)")
        else:
            label_NP.config(text="Positive(+)")

    
btn_browse = Button(top_text_frame , text = 'browse',command = getPathBtn)
btn_browse.pack()


window.mainloop()

C:/Users/SUNNY/Sunny storage/work/glaucoma/program/we_want_some_eyes_donut/RAW_FUNDUS_INPUT/271_086.jpg
247.0
(500, 500)
(500, 500)
C:/Users/SUNNY/Sunny storage/work/glaucoma/program/we_want_some_eyes_donut/RAW_FUNDUS_INPUT/267_008.jpg
296.0
(500, 500)
(500, 500)


<Figure size 432x288 with 0 Axes>